# Exemplo simples de FEM com Ogum

Este notebook demonstra como gerar uma malha unitaria, simular a densificação e visualizar o resultado.

In [ ]:
from ogum.fem_interface import create_unit_mesh, densify_mesh
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv


## Geração da malha

In [ ]:
mesh = create_unit_mesh(0.2)


## Histórico de temperatura

In [ ]:
times = np.linspace(0, 60 * 60, num=61)
temps = np.linspace(25, 1500, num=61)
temperature_history = list(zip(times, temps))


## Execução da simulação

In [ ]:
densities = densify_mesh(mesh, temperature_history, solver_options={'Ea': 3.1e5, 'A': 1e6, 'n': 1.5})


## Visualização 3D com PyVista

In [ ]:
try:
    from dolfinx import plot
    cells, cell_types, geometry = plot.create_vtk_mesh(mesh, mesh.topology.dim)
    grid = pv.UnstructuredGrid(cells, cell_types, geometry)
    grid.cell_data['density'] = densities
    pv.set_plot_theme('document')
    plotter = pv.Plotter()
    plotter.add_mesh(grid, scalars='density', cmap='viridis', show_edges=True)
    plotter.add_scalar_bar('Density')
    plotter.show()
except Exception as exc:
    print('Falha no PyVista:', exc)
    triangles = mesh.topology.connectivity(mesh.topology.dim, 0).array.reshape(-1, 3)
    x = mesh.geometry.x[:, 0]
    y = mesh.geometry.x[:, 1]
    plt.tripcolor(x, y, triangles, densities, cmap='viridis', shading='flat')
    plt.colorbar(label='Density')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.gca().set_aspect('equal')
    plt.show()
